In [2]:
import pandas as pd
import numpy as np
import json
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from pprint import pprint

In [145]:
json_file = '../uber_data/san_francisco_censustracts.json'
json_data=open(json_file).read()
data = json.loads(json_data)
#pprint(data)

In [146]:
data['features'][1]['geometry']['coordinates'][0][0]

[[-122.224626, 37.465065],
 [-122.223616, 37.469312],
 [-122.22358, 37.469466],
 [-122.221763, 37.473925],
 [-122.221776, 37.476151],
 [-122.220521, 37.478913],
 [-122.21764, 37.477094],
 [-122.214839, 37.47532],
 [-122.215441, 37.474855],
 [-122.216813, 37.472842],
 [-122.215655, 37.471997],
 [-122.21211, 37.469437],
 [-122.215954, 37.464865],
 [-122.21701, 37.465068],
 [-122.218533, 37.462448],
 [-122.218376, 37.463114],
 [-122.221052, 37.462831],
 [-122.224778, 37.464346],
 [-122.224626, 37.465065]]

In [147]:
np.mean(data['features'][1]['geometry']['coordinates'][0][0], axis=0)

array([-122.21940574,   37.46958495])

In [148]:
data['features'][0]['properties']['MOVEMENT_ID']

'1'

In [149]:
data['features'][189]['properties']['DISPLAY_NAME']

'South San Joaquin Rocks Lookout Road, Coalinga'

In [163]:
data['features'][2]

{'geometry': {'coordinates': [[[[-121.679781, 37.158594],
     [-121.671983, 37.164063],
     [-121.670184, 37.165278],
     [-121.66749, 37.167032],
     [-121.663253, 37.162396],
     [-121.655618, 37.156437],
     [-121.645075, 37.145542],
     [-121.640905, 37.141237],
     [-121.64082, 37.14115],
     [-121.642424, 37.139813],
     [-121.64338, 37.13904],
     [-121.65269, 37.13173],
     [-121.66269, 37.14148],
     [-121.66333, 37.14345],
     [-121.66718, 37.14628],
     [-121.677079, 37.155975],
     [-121.679781, 37.158594]]]],
  'type': 'MultiPolygon'},
 'properties': {'DISPLAY_NAME': '18300 Sutter Boulevard, Morgan Hill',
  'MOVEMENT_ID': '3'},
 'type': 'Feature'}

In [204]:
n = len(data['features'])
df_data = []
for i in range(0, n):
    #print(i)
    mov_id = data['features'][i]['properties']['MOVEMENT_ID']
    location = np.mean(data['features'][i]['geometry']['coordinates'][0][0], axis=0)
    disp_name = data['features'][i]['properties']['DISPLAY_NAME']
    df_data.append({"movement_id": mov_id, "location": location, "display_name": disp_name})
df_data[:5]

[{'display_name': 'Sargent Creek, San Ardo',
  'location': array([-120.78868462,   36.07162944]),
  'movement_id': '1'},
 {'display_name': '400 Northumberland Avenue, Redwood Oaks, Redwood City',
  'location': array([-122.21940574,   37.46958495]),
  'movement_id': '2'},
 {'display_name': '18300 Sutter Boulevard, Morgan Hill',
  'location': array([-121.66021547,   37.15047594]),
  'movement_id': '3'},
 {'display_name': '2700 Stoughton Way, Sheffield, Sacramento',
  'location': array([-121.3428459 ,   38.57307385]),
  'movement_id': '4'},
 {'display_name': '3200 Huntsman Drive, Rosemont Park, Sacramento',
  'location': array([-121.36171355,   38.5544347 ]),
  'movement_id': '5'}]

In [210]:
json_df = pd.DataFrame.from_records(df_data)
json_df.head()

,display_name,location,movement_id
0,"Sargent Creek, San Ardo","[-120.788684621, 36.0716294437]",1
1,"400 Northumberland Avenue, Redwood Oaks, Redwo...","[-122.219405737, 37.4695849474]",2
2,"18300 Sutter Boulevard, Morgan Hill","[-121.660215471, 37.1504759412]",3
3,"2700 Stoughton Way, Sheffield, Sacramento","[-121.3428459, 38.57307385]",4
4,"3200 Huntsman Drive, Rosemont Park, Sacramento","[-121.36171355, 38.5544347]",5


In [207]:
json_df.to_csv('../uber_data/Geo_Data.csv', index=False)

<h2>Movement data CSV</h2>

In [3]:
csv_file = '../uber_data/san_francisco-censustracts-2017-4-All-MonthlyAggregate.csv'

In [4]:
df = pd.read_csv(csv_file)
df = df[df['month'] == 12]

In [5]:
df_edgelist = df[['sourceid', 'dstid', 'mean_travel_time']]
df_edgelist.head()

,sourceid,dstid,mean_travel_time
5,1938,1952,596.69
6,1747,629,1454.81
8,1872,1919,1227.01
9,1871,1929,1005.54
10,81,2319,474.53


In [6]:
len(df_edgelist['sourceid'].unique())

1852

In [7]:
len(df_edgelist['dstid'].unique())

1885

In [140]:
df_edgelist.groupby(['sourceid','dstid'], as_index=False).size();

In [139]:
weights = df_edgelist[['mean_travel_time']].values.ravel()
row = df_edgelist[['sourceid']].values.ravel()
col = df_edgelist[['dstid']].values.ravel()
#print(len(row))
#print(len(col))
#n = len(np.unique(np.append(row, col)))

In [138]:
print(row)
print(col)
print(len(weights))

[1938 1747 1872 ..., 1311 1304 1333]
[1952  629 1919 ..., 2466 2536 2246]
538998


In [96]:
coo = coo_matrix((weights, (row, col)))
csr = coo.tocsr()
csr[csr == 0] = np.nan

/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:214: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  ", try using != instead.", SparseEfficiencyWarning)
/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:742: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [99]:
no_duplicate = np.nanmean(np.array([csr.toarray(), csr.T.toarray()]), axis=0)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


In [100]:
nonzero_edges = np.argwhere(no_duplicate)
len(nonzero_edges)

7344100

In [104]:
edge_data = []
for (i,j) in nonzero_edges:
    edge_data.append({"V1":i, "V2":j, "Weight":no_duplicate[i,j]})
edge_data[:5]

[{'V1': 0, 'V2': 0, 'Weight': nan},
 {'V1': 0, 'V2': 1, 'Weight': nan},
 {'V1': 0, 'V2': 2, 'Weight': nan},
 {'V1': 0, 'V2': 3, 'Weight': nan},
 {'V1': 0, 'V2': 4, 'Weight': nan}]

In [114]:
edge_df = pd.DataFrame.from_records(edge_data)
edge_df = edge_df.dropna()
edge_df.head()

,V1,V2,Weight
5429,2,9,2787.965
5440,2,20,2784.245
5481,2,61,1374.620
5499,2,79,2876.640
5502,2,82,2132.185


In [120]:
len(set.intersection(set(edge_df['V1']), set(edge_df['V2'])))

1887

In [202]:
edge_df.columns

Index(['V1', 'V2', 'Weight'], dtype='object')

In [134]:
def edgelist(df):
    a = df.values
    c = df.columns
    n = len(c)
    #print(n)
    c_ar = np.array(c)
    out = np.empty((n, n, 2), dtype=c_ar.dtype)

    out[...,0] = c_ar[:,None]
    out[...,1] = c_ar

    #mask = ~np.eye(n,dtype=bool)
    mask = ~np.triu(np.ones((n,n), dtype=bool))
    df_out = pd.DataFrame(out[mask], columns=[['V1','V2']])
    df_out['Weight'] = a[mask]
    return df_out

In [195]:
edgelist_matrix = edgelist(pd.DataFrame(no_duplicate))
edgelist_matrix = edgelist_matrix.dropna()

In [143]:
edgelist_matrix.to_csv('../uber_data/movement_edgelist.txt', sep=' ', header=False, index=False)